In [ ]:
import folium
import h3
from IPython.display import display

from srai.datasets import PortoTaxiDataset

In [ ]:
porto_taxi = PortoTaxiDataset()

In [ ]:
type(porto_taxi.train_gdf), type(porto_taxi.test_gdf)

Get data using .load() method -> Default config (Travel time estimation)

In [ ]:
ds = porto_taxi.load()
ds.keys()

In [ ]:
type(porto_taxi.train_gdf), type(porto_taxi.test_gdf)

In [ ]:
ds["train"].head()

In [ ]:
ds["test"].head()

Getting h3 trajectories with target valeus

In [ ]:
porto_taxi.resolution

In [ ]:
train_h3, test_h3 = porto_taxi.get_h3_with_labels()

In [ ]:
train_h3

In [ ]:
test_h3

Get Human Mobility Prediction (HMP) data

In [ ]:
ds = porto_taxi.load(version="HMP")

In [ ]:
ds["train"].head()

Creating your own train_test split based on trajectory duration (version TTE) or length version (HMP). Passing resolution parameter is neccessary for generation a trajectory in h3 style.

In [ ]:
ds = porto_taxi.load(version="all", resolution=9)
ds.keys()

In [ ]:
ds["train"].head()

In [ ]:
train, test = porto_taxi.train_test_split_bucket_trajectory(
    trajectory_id_column="trip_id", task="HMp", test_size=0.2, bucket_number=7
)

In [ ]:
len(train), len(test)

In [ ]:
def visualize_h3_trajectories(h3_sequences, map_center=(41.14075, -8.61029), zoom_start=12):
    """
    Visualize H3 sequences on a Folium map.

    Args:
        h3_sequences (List[List[str]]): A list of H3 sequences (trajectories).
        map_center (Tuple[float, float]): Center of the map (lat, lon).
        zoom_start (int): Initial zoom level.
    """
    m = folium.Map(location=map_center, zoom_start=zoom_start, tiles="cartodbpositron")

    colors = ["red", "blue", "green", "purple", "orange", "darkred", "lightblue"]

    for i, sequence in enumerate(h3_sequences):
        color = colors[i % len(colors)]

        for h3_id in sequence:
            boundary = h3.cell_to_boundary(
                h3_id,
            )
            folium.Polygon(
                locations=boundary, color=color, weight=2, fill=True, fill_opacity=0.3
            ).add_to(m)

    return m


h3_sequences = train["h3_sequence"].tolist()
map_ = visualize_h3_trajectories(h3_sequences[0:10])  # visualize first 10 for speed
display(map_)